# Question 1
## Part 1.1 Implement an agent-based model 

You can use the following template or feel free to implement your own program. Please refer to the handout for details about parameters settings. 

In [293]:
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

#### Define Agent class

In [336]:
class Agent:
    '''
    Defines an Agent for this model.
    
    Allowable statuses: (R) Recovered
                        (S) Susceptible to Infection
                        (I) Infected
                        (Q) Quarantined
                        
    Asymptomatic:  True if agent is asymptomatic when infected
    Distancing: True if agent is physically distancing
    Mask: True if agent is wearing a mask
    '''
    statuses = ['R', 'S', 'I', 'Q'] # allowable statuses
    days_infected = 0    # How long agent has been infected, if currently infected
    
    def __init__(self, 
                 position, 
                 status,  
                 mask, 
                 distancing):
        '''
        Initialize an agent.
        '''
        # position (x,y)
        self.position = position

        # status
        if status in self.statuses:
            self.status = status
        else:
            raise ValueError('status: ' + status + ' is not valid.')

        # possibly asymptomatic if infected
        if status == 'I':
            if self.is_asymptomatic():
                 self.asymptomatic = True
            else:
                self.asymptomatic = False
        else:
            self.asymptomatic = False 
                            
        self.mask = mask
        self.distancing = distancing 
        
    
    def is_event(self, num_event, num_outcomes):
        '''
        If an event has a 25% chance, num_event = 1 and num_outcomes = 4
        Returns true if event is selected from outcomes, False otherwise.
        '''
        return random.randint(1,num_outcomes) <= num_event
    
    
    def is_infected(self, adjacent_agents):
        '''
        Returns True if the agent is now infected, given whether
        an adjacent agent is infected or masked and whether the 
        agent is masked. False otherwise.
        
        Check is performed for each agent adjacent to this agent. 
        ''' 
        infected = False
        
        # perform check for all adjacent agents
        for adjacent in adjacent_agents:
        
            # chance of infection if an adjacent agent is infected
            if adjacent.status == 'I':

                # infection probability depends on mask status of both
                if self.mask and adjacent.mask: 

                    if self.is_event(1,10000): # 0.01% chance
                        infected = True

                elif self.mask or adjacent.mask: 

                    if self.is_event(1,100):  # 1% chance
                        infected = True

                else:  

                    if self.is_event(1,4):   # 25% chance
                        infected = True
            
        return infected
            
    
    def is_asymptomatic(self):
        '''
        Returns True if agent is asymptomatic, False otherwise.
        '''
        return self.is_event(1,5) # 20% chance of being asymptomatic
        
    
    def will_quarantine(self):
        '''       
        Returns True if the agent has been infected for 
        at least 3 days and is symptomatic, False otherwise.
        '''       
        return self.days_infected > 2 and not self.asymptomatic

        
    
    def infection_over(self):
        '''
        Returns True if the agent has been infected for at
        least 15 days, False otherwise
        '''
        return self.days_infected > 14
                
    
    def update_status(self, adjacent_agents):
        '''
        Checks whether modification to the agent's status 
        is needed, and updates the status as necessary.
        '''
        
        if self.status == 'S': # not yet infected
            
            if self.is_infected(adjacent_agents):
                self.status = 'I'
                
        elif self.status == 'I': # already infected
            
            if self.will_quarantine():
                self.status = 'Q'
        
            if self.infection_over():
                self.status = 'R'
                
        elif self.status == 'Q': # already in quarantine
            
            if self.infection_over():
                self.status = 'R'
            
            
    def update_agent(self, adjacent_agents):
        '''
        Updates agent metrics based on adjacent agents.
        Does not update position.
        '''
        
        # update days infected if already infected
        if self.status == 'I' or self.status == 'Q':
            self.days_infected += 1 # day passed since infection started            
        
        # update status
        status_before = self.status
        self.update_status(adjacent_agents)
        
         # if now infected, determine if asymptomatic
        if status_before != 'I' and self.status == 'I':
            self.asymptomatic = self.is_asymptomatic()
            
        elif status_before != 'R' and self.status == 'R':
            
            # reset metrics related to infection
            self.days_infected = 0
            self.asymptomatic = False
            
    
    def __str__(self):  
        return  'Agent{'+'position:'+ str(self.position)+', status:'+ str(self.status)+', mask:'+ str(self.mask)+', distancing:'+ str(self.distancing)+', days_infected:'+ str(self.days_infected)+', asymptomatic:'+ str(self.asymptomatic)+'}'      
    
    
    def has_died(self):
        '''
        Return True if agent dies due to infection, False otherwise.
        '''  
        return self.status == 'I' and self.is_event(2,1000) #0.2% chance
    

#### Test Agent class

In [295]:
print("Test constructor")
a1 = Agent((1,1), 'I', mask=True, distancing=True)
print(a1)

# check provided parameters
assert a1.position == (1,1)
assert a1.status == 'I'
assert a1.mask == True
assert a1.distancing == True
assert a1.days_infected == 0


# check derived parameters
num = 100000
counter = 0
for i in range(num):
    a1 = Agent((1,1), 'I', mask=True, distancing=True)
    if a1.asymptomatic == True:
        counter += 1
assert round(counter/num,2) == 0.20

Test constructor
Agent{position:(1, 1), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}


In [296]:
print('Test is_event()')
a1 = Agent((1,1), 'I', mask=True, distancing=True)

num = 100000
counter = 0
for i in range(num):
    if a1.is_event(1,10000):
        counter += 1
assert round(counter/num,4) == 0.0001, 'Failed with ' + str(round(counter/num,4))

counter = 0
for i in range(num):
    if a1.is_event(1,100):
        counter += 1
assert round(counter/num,2) == 0.01

counter = 0
for i in range(num):
    if a1.is_event(1,4):
        counter += 1
assert round(counter/num,2) == 0.25

counter = 0
for i in range(num):
    if a1.is_event(2,1000):
        counter += 1
assert round(counter/num,3) == 0.002, 'Failed with ' + str(round(counter/num,3))

Test is_event()


In [297]:
print('Test is_asymptomatic()')

counter = 0
for i in range(num):
    if a1.is_asymptomatic():
        counter += 1
assert round(counter/num,2) == 0.20

Test is_asymptomatic()


In [298]:
print('Test will_quarantine()')

a2 = Agent((0,1), 'I', mask=True, distancing=True)

# possibility of quarantine when symptomatic
a2.asymptomatic = False

a2.days_infected = 2
assert a2.will_quarantine() == False

a2.days_infected = 3
assert a2.will_quarantine() == True

a2.days_infected = 4
assert a2.will_quarantine() == True


# when asymptomatic, will not quarantine
a2.asymptomatic = True
a2.days_infected = 2
assert a2.will_quarantine() == False

a2.days_infected = 3
assert a2.will_quarantine() == False

a2.days_infected = 4
assert a2.will_quarantine() == False


Test will_quarantine()


In [299]:
print('Test infection_over()')
a3 = Agent((0,1), 'I', mask=True, distancing=True)
a3.days_infected = 13
assert a3.infection_over() == False

a3.days_infected = 14
assert a3.infection_over() == False

a3.days_infected = 15
assert a3.infection_over() == True


Test infection_over()


In [300]:
print('Test is_infected()')
a1 = Agent((1,1), 'S', mask=True, distancing=True)
a2 = Agent((0,1), 'I', mask=True, distancing=True)

# test 1 adjacent, 1 infected, both masked
a1.mask = True
a2.mask = True
num = 100000
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,4) == 0.0001, round(counter/num,4)

# test 1 adjacent, 1 infected, only agent masked
a1.mask = True
a2.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,2) == 0.01

# test 1 adjacent, 1 infected, only adjacent masked
a1.mask = False
a2.mask = True
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,2) == 0.01

# test 1 adjacent, 1 infected, no masks
a1.mask = False
a2.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,2) == 0.25

Test is_infected()


In [301]:
print('Test is_infected()')
a3 = Agent((1,2), 'R', mask=True, distancing=True)

# test 2 adjacent, 1 infected, all masked
a1.mask = True
a2.mask = True
a3.mask = True
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
assert round(counter/num,4) == 0.0001

# test 2 adjacent, 1 infected, only agent masked
a1.mask = True
a2.mask = False
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
assert round(counter/num,2) == 0.01

# test 2 adjacent, 1 infected, no one masked
a2.mask = False
a3.mask = False
a1.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
assert round(counter/num,2) == 0.25

Test is_infected()


In [302]:
print('Test is_infected()')
# test 2 adjacent, 2 infected, all masked
a3.status = 'I'
a1.mask = True
a2.mask = True
a3.mask = True
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,4))
assert round(counter/num,4) > 0.0001, 'Failed with ' + str(round(counter/num,4))

# test 2 adjacent, 2 infected, agent masked
a3.status = 'I'
a1.mask = True
a2.mask = False
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,2))
assert round(counter/num,2) > 0.01

# test 2 adjacent, 2 infected, agent masked, one adjacent masked
a3.status = 'I'
a1.mask = True
a2.mask = True
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,2))
assert round(counter/num,2) >= 0.01

# test 2 adjacent, 2 infected, no one masked
a3.status = 'I'
a1.mask = False
a2.mask = False
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,2))
assert round(counter/num,2) > 0.25

Test is_infected()
0.0002
0.02
0.01
0.44


In [303]:
print('Test update_status()')
a1 = Agent((1,1), 'R', mask=True, distancing=True)
a2 = Agent((0,1), 'I', mask=False, distancing=True)

# test R status
print('Test R status...')
a1.update_status([a2])
assert a1.status == 'R'

# test I status
print('Test I status...')

a1.status = 'I'
a1.days_infected = 0 # test not yet quarantined
a1.asymptomatic = True
a1.update_status([a2])  
assert a1.status == 'I'


a1.status = 'I'
a1.days_infected = 3 # test change to quarantined
a1.asymptomatic = False
a1.update_status([a2]) 
assert a1.status == 'Q'

a1.status = 'I'
a1.days_infected = 15 # test change to recovered
a1.asymptomatic = False
a1.update_status([a2]) 
assert a1.status == 'R'

# test Q status
print('Test Q status...')
a1.status = 'Q'
a1.days_infected = 15 # test change to recovered
a1.asymptomatic = False
a1.update_status([a2]) 
assert a1.status == 'R'

# test S status
print('Test S status...')
a1.status = 'S'
a1.days_infected = 0
a1.asymptomatic = False
num = 10000
counter = 0
for i in range(num):
    a1.status = 'S'
    a1.update_status([a2])
    if a1.status == 'I':
        counter += 1
assert round(counter/num,2) == 0.01

Test update_status()
Test R status...
Test I status...
Test Q status...
Test S status...


In [304]:
print('Test update_agent()')

a1 = Agent((1,1), 'R', mask=False, distancing=True)
a2 = Agent((0,1), 'I', mask=False, distancing=True)


# check days_infected
a1.status = 'I'
a1.update_agent([a2])
assert a1.days_infected == 1

# check status if now recovered
# check metrics if now recovered
a1.status = 'I'
a1.days_infected = 14
a1.update_agent([a2])
assert a1.status == 'R'
assert a1.days_infected == 0
assert a1.asymptomatic == False

# check status if now recovered
# check metrics if now recovered
a1.status = 'Q'
a1.days_infected = 14
a1.update_agent([a2])
assert a1.status == 'R'
assert a1.days_infected == 0
assert a1.asymptomatic == False
    

# check asymptomatic if now infected
a1.status = 'S'
a1.days_infected = 0
a1.asymptomatic = False
a1.update_agent([a2])

num = 100000
counter = 0
for i in range(num):
    a1.status = 'S'
    a1.asymptomatic = False
    a1.update_agent([a2])
    if a1.asymptomatic == True:
        counter += 1
assert round(counter/num,2) == 0.05

Test update_agent()


In [305]:
print('Test has_died()')
num = 100000

# check for infected status
a2 = Agent((0,1), 'I', mask=False, distancing=True)
counter = 0
for i in range(num):
    if a2.has_died():
        counter += 1
assert round(counter/num,3) == 0.002


# check if status not infected
a3 = Agent((0,1), 'S', mask=False, distancing=True)
counter = 0
for i in range(num):
    if a3.has_died():
        counter += 1
assert round(counter/num,1) == 0.0

Test has_died()


In [306]:
print('All tests pass for Agent class.')

All tests pass for Agent class.


#### Define AgentGenerator class

In [307]:
class AgentGenerator:
    '''
    Defines functionality for generating the required 
    number of Agents with the requisite properties
    (i.e. the correct number of infected, distancing, masked, vaccinated).
    
    Mask and distancing properties are set independent of each other.
    Agents are randomly chosen among infected and vaccinated when deciding which Agents are masked.
    Agents are randomly chosen among infected and vaccinated when deciding which Agents are distancing.
    '''
    
    
    def __init__(self, m, num_infected, percent_distancing, percent_mask, percent_vaccinated = 0.0):
        '''
        Initialize the world and agents in the world. 

        Parameters:
        n: size of the world
        m: number of agents 
        '''        
        # capture parameters
        self.m = m
        self.num_infected = num_infected

        # set discrete numbers for distancing, mask, vaccinated
        self.num_distancing = int(round(percent_distancing*m))
        self.num_mask = int(round(percent_mask*m))
        self.num_vaccinated = int(round(percent_vaccinated*m))
        
        # validation checks        
        if self.num_infected + self.num_vaccinated > m:
            raise ValueError('not allowed to have (infected + vaccinated) > total number of agents')
    
    
    def is_infected(self, current_infected):
        '''
        Returns True if there should be more infected agents 
        (i.e. this agent is selected as infected). Returns False otherwise.
        '''
        return current_infected < self.num_infected
    
    
    def is_vaccinated(self,infected,current_vaccinated):
        '''
        Returns True if the agent is not infected and there should be more vaccinated agents 
        (i.e. this agent is selected as vaccinated). 
        Returns False otherwise.
        '''
        if infected:
            return False # can't be both infected and vaccinated
        else: 
            return current_vaccinated < self.num_vaccinated
    
    
    def get_status(self, infected, vaccinated):
        '''
        Used to help determine which status should be chosen.
        
        If infected and not vaccinated, returns 'I'.
        If neither infected nor vaccinated, returns 'S'.
        If vaccinated, returns 'R'.
        '''
        if infected and vaccinated:
            raise ValueError('Agent is not allowed to be both infected and vaccinated.')
        
        status = 'S'
        if infected:
            status = 'I'
        elif vaccinated:
            status = 'R'
        
        return status
    

    def select_indexes(self, num):
        '''
        Selects num random positions between 0 and m-1. 
        '''
    
        chosen = list()
        for i in range(num):
            
            position = random.randint(0,self.m-1)
            while position in chosen:
                
                # try again
                position = random.randint(0,self.m-1)
            
            chosen.append(position)
        return chosen


    def generate_agents(self):
        '''
        Creates and returns a list of m agents, each created with 
        a randomly chosen position from the n x n grid, with the 
        required number of agents selected as infected and vaccinated.
        
        After setting status, randomly selects agents to be masked.
        Separately from mask, randomly selects agents to be distancing.
        '''
        
        # create agents
        agents = list()
        
        # set counters
        current_infected = 0
        current_vaccinated = 0
        
        # randomly select indexes for mask and distancing
        mask_indexes = self.select_indexes(self.num_mask)
        distancing_indexes =  self.select_indexes(self.num_distancing)
        
        for i in range(self.m):
        
            # select if agent is infected
            infected = self.is_infected(current_infected)
            if infected:
                current_infected += 1
            
            # select if agent is vaccinated
            vaccinated = self.is_vaccinated(infected, current_vaccinated)
            if vaccinated:
                current_vaccinated += 1
                
            # set status
            status = self.get_status(infected, vaccinated)

            # set values for mask and distancing
            mask = i in mask_indexes
            distancing = i in distancing_indexes
                
            # position will be set later
            new_agent = Agent((-1,-1), status, mask=mask, distancing=distancing)
            agents.append(new_agent)          
        
        return agents
    

#### Testing AgentGenerator class

In [308]:
print('Test constructor')

# test version with defined percent_vaccinated
ag = AgentGenerator(100,10,0.25,0.35,0.45)
assert ag.m == 100
assert ag.num_infected == 10
assert ag.num_distancing == 25
assert ag.num_mask == 35
assert ag.num_vaccinated == 45

# test version without defined percent_vaccinated
ag = AgentGenerator(100,10,0.25,0.35)
assert ag.m == 100
assert ag.num_infected == 10
assert ag.num_distancing == 25
assert ag.num_mask == 35
assert ag.num_vaccinated == 0

Test constructor


In [309]:
print('Test is_infected()')

ag = AgentGenerator(100,1,0.25,0.35,0.45)
current_infected = 0
infected = ag.is_infected(current_infected)
assert infected == True

current_infected = 1
infected = ag.is_infected(current_infected)
assert infected == False

Test is_infected()


In [310]:
print('Test is_vaccinated()')

ag = AgentGenerator(100,1,0.25,0.35,0.45)

# infected should not become vaccinated
current_vaccinated = 0
infected = True
vaccinated = ag.is_vaccinated(infected,current_vaccinated)
assert vaccinated == False


# remaining should become vaccinated
current_vaccinated = 0
infected = False
vaccinated = ag.is_vaccinated(infected,current_vaccinated)
assert vaccinated == True

# full should not become vaccinated
current_vaccinated = 45
infected = False
vaccinated = ag.is_vaccinated(infected,current_vaccinated)
assert vaccinated == False



Test is_vaccinated()


In [311]:
print('Test get_status()')
ag = AgentGenerator(100,1,0.25,0.35,0.45)

# infected
infected = True
vaccinated = False
status = ag.get_status(infected, vaccinated)
assert status == 'I'

# vaccinated
infected = False
vaccinated = True
status = ag.get_status(infected, vaccinated)
assert status == 'R'

# susceptible
infected = False
vaccinated = False
status = ag.get_status(infected, vaccinated)
assert status == 'S'


# error handling
try:
    infected = True
    vaccinated = True
    status = ag.get_status(infected, vaccinated)
    assert False
except ValueError:
    assert True

Test get_status()


In [312]:
print('Test select_indexes()')

m = 100
ag = AgentGenerator(m,1,0.25,0.35,0.45)

num_positions = 25
positions = ag.select_indexes(num_positions)

assert len(positions) == num_positions
assert len(set(positions)) == num_positions

counter = 0
for p in positions:
    if p >= m or p < 0: # should be within [0...m-1] index positions
        counter += 1
assert counter == 0
print(positions)

Test select_indexes()
[19, 54, 55, 10, 52, 13, 87, 44, 2, 35, 71, 56, 39, 92, 45, 61, 77, 78, 76, 38, 0, 33, 27, 84, 51]


In [313]:
print('Test create_agents()')
m = 100
ag = AgentGenerator(m,10,0.25,0.35,0.45)
agents = ag.generate_agents()

assert len(agents) == m

infected_counter = 0
vaccinated_counter = 0
susceptible_counter = 0
mask_counter = 0
distancing_counter = 0

for agent in agents:
    if agent.status == 'I':
        infected_counter += 1
    elif agent.status == 'R':
        vaccinated_counter += 1
    elif agent.status == 'S':
        susceptible_counter += 1
        
    if agent.mask:
        mask_counter += 1
    
    if agent.distancing:
        distancing_counter += 1

assert infected_counter == 10, infected_counter
assert distancing_counter == 25, distancing_counter
assert mask_counter == 35, mask_counter
assert vaccinated_counter == 45, vaccinated_counter
assert susceptible_counter == m - infected_counter - vaccinated_counter, susceptible_counter


Test create_agents()


In [314]:
print('All tests pass for AgentGenerator class.')

All tests pass for AgentGenerator class.


#### Define AgentMover class
Class must consider how wrapping around the torus grid works, as shown below.

![02718-HW4-Wrap.png](images/02718-HW4-Wrap.png)

The grid space which must be considered while moving an agent can be limited to a region 3 units from the agent's current position, as shown below.

![02718-HW4-Moves.png](images/02718-HW4-Moves.png)

Available moves may be restricted depending on whether nearby agents are distancing and whether this agent is distancing. The restrictions turn out to be exactly the same.

Restrictions if nearby agents are distancing
![02718-HW4-OtherDistancing.png](images/02718-HW4-OtherDistancing.png)

Restrictions if this agent is distancing
![02718-HW4-SelfDistancing.png](images/02718-HW4-SelfDistancing.png)

In [345]:
class AgentMover:
    
    # allowable moves
    directions = [[-1, 0], # left
                  [ 1, 0], # right
                  [ 0,-1], # up
                  [ 0, 1], # down
                  [-1,-1], # left & up
                  [-1, 1], # left & down
                  [ 1,-1], # right & up
                  [ 1, 1]] # right & down
    
    # all moves which extend 3 positions from current position (0,0)
    directions_dia_3 = [[i,j] for i in range(-3,4) for j in range (-3,4) if (i,j) != (0,0)]
    
    # all moves which extend 2 positions from current position (0,0)
    directions_dia_2 = [[i,j] for i in range(-2,3) for j in range (-2,3) if (i,j) != (0,0)]
    
    def __init__(self, n):
        self.n = n
        

    def get_random_position(self, selected):
        '''
        Returns a random (i,j) position in an n x n grid 
        which has not yet been selected.
        '''
        
        # try random position
        i = random.randint(0,self.n-1)
        j = random.randint(0,self.n-1)
        
        while (i,j) in selected:

            # keep trying if position has already been chosen
            i = random.randint(0,self.n-1)
            j = random.randint(0,self.n-1)
            
        return (i,j)
    
    
    def split_agents(self, agents):
        '''
        Splits agents into those which are distancing and other. 
        Returns tuple of two lists:(distancing agents, other agents)
        '''
        
        # position distancing agents first
        distancing_agents = list()
        other_agents = list()
        
        for agent in agents:
            if agent.distancing:
                distancing_agents.append(agent)
            else:
                other_agents.append(agent)
                
        return distancing_agents, other_agents
    
        
    def get_x(self, x_curr, change):
        '''
        Calculates next x coordinate, wrapping around torus 
        grid if position would move off the top or bottom.
        '''
        
        x_result = x_curr + change
        if x_result < 0:
            x_next = self.n + x_result
        elif x_result > self.n-1:
            x_next = x_result % self.n
        else:
            x_next = x_result
                
        return x_next
 

    def get_y(self, y_curr, change):
        '''
        Calculates next y coordinate, wrapping around torus 
        grid if position would move off the left or right.
        '''                       
        return self.get_x(y_curr, change) # works the same as x changes

    
    def get_adj_positions(self, position):
        '''
        Returns a list of all 8 positions which surround the given position, 
        wrapping around the torus grid when necessary.
        '''

        adjacent = set()
        
        x_curr = position[0]
        y_curr = position[1]
        
        for x_change,y_change in self.directions:
            
            # get resulting coordinates after change
            x_next = self.get_x(x_curr, x_change)
            y_next = self.get_y(y_curr, y_change)
        
            adjacent.add((x_next, y_next))
        
        return adjacent
    
    
    def get_search_space(self, position, directions_set):
        '''
        Returns a list of all positions which surround the given position, 
        representing all moves within the directions set, and 
        wrapping around the torus grid when necessary.
        
        For a directions set of diameter 3, there are 48 positions.
        For a directions set of diameter 2, there are 24 positions.
        For a directions set of diameter 1, there are 8 positions.
        '''
        
        adjacent = set()
        
        x_curr = position[0]
        y_curr = position[1]
        
        for x_change,y_change in directions_set: 
                
            # get resulting coordinates after change
            x_next = self.get_x(x_curr, x_change)
            y_next = self.get_y(y_curr, y_change)

            adjacent.add((x_next, y_next))
        
        return adjacent
    
    
    def positions_available(self, unavailable, agents, positioned):
        '''
        Returns True if there is at least one positions 
        available for each agent not yet positioned.
        '''
        
        num_agents_left = len(agents) - positioned
        num_positions_left = self.n * self.n - len(unavailable)
        return num_positions_left >= num_agents_left
    
    
    def position_distancing_agents(self, distancing_agents):
        '''
        Attempts to find unique positions for given list of distancing agents 
        such that every agent maintains at least one position between itself
        and every other agent. Returns (True, unavailable) if such a positioning 
        is found. Returns (False, unavailable) otherwise. unavailable is a
        set containing all positions which can not be selected for agents.
        '''
                   
        # reset for new attempt
        unavailable = set()
        num_positioned = 0

        for agent in distancing_agents: 
            
            # if there aren't enough available positions left, start over
            if not self.positions_available(unavailable, distancing_agents, num_positioned):
                return (False, unavailable)           
        
            # choose available position
            position = self.get_random_position(unavailable)
            agent.position = position

            # update available positions
            curr_unavailable = self.get_adj_positions(position) # positions surrounding are not available
            curr_unavailable.add(position)                      # position taken is not available
            unavailable = unavailable.union(curr_unavailable)   # update full list
        
            # update counter
            num_positioned += 1
        
        # all agents were positioned successfully
        return (True, unavailable)
    
    
    def position_remaining_agents(self, agents, unavailable):
        '''
        Positions all agents without distancing requirements.
        Returns True if agents were positioned successfully,
        False otherwise.
        '''
        
        num_positioned = 0
        for agent in agents:
            
            # if there aren't enough available positions left, start over
            if not self.positions_available(unavailable, agents, num_positioned):
                return False
        
            # choose available position
            position = self.get_random_position(unavailable)
            agent.position = position

            # update available positions                     
            unavailable.add(position)
        
            # update counter
            num_positioned += 1
        
        # all agents were positioned successfully
        return True
        

    def position_agents(self, agents):
        '''
        Positions given agents on n x n grid, following distancing constraints
        for distancing agents. Raises ValueError if no viable positioning
        is found. Returns True if positioning is found.
        '''
        
        # split agents into distancing and other
        distancing_agents, other_agents = self.split_agents(agents)
        
        # attempt to position distancing agents first
        attempt_counter_inner = 0
        attempt_counter_outer = 0
        d_success = False
        all_success = False
        unavailable = set()
        
        while all_success == False:
        
            # try to position distancing agents
            while d_success == False:

                d_success, unavailable = self.position_distancing_agents(distancing_agents)
                
                attempt_counter_inner += 1
                if attempt_counter_inner >= 1000:
                    raise ValueError('Unable to find positions for all distancing agents')


            # position remaining agents
            all_success = self.position_remaining_agents(other_agents, unavailable)
            
            attempt_counter_outer += 1
            if attempt_counter_outer >= 1000:
                raise ValueError('Unable to find positions for other agents')

 
        # all agents positioned
        return True
     
 
    def find_nearby_agents(self, agents, position):
        '''
        Finds all agents within 3 spaces of this agent in every direction.
        This reduces the search space for available positions to only
        those agents which may be affected by this agent after a move.
        '''
        # determine positions within 3 spaces of this agent
        search_space = self.get_search_space(position, self.directions_dia_3)
        
        # find all agents within that search space
        nearby = list()
        for agent in agents:
            if agent.position in search_space:
                nearby.append(agent)
        
        return nearby
    
    
    def remove_position_if_occupied(self, other, avail):
        '''
        If position of other agent is in available positions,
        removes it.
        '''
        
        if other.position in avail:
            avail.remove(other.position)
     
    
    def remove_positions_distancing_others(self, other, avail):
        '''
        If other agent is distancing, removes all positions
        adjacent to other agent from available positions.
        '''
        
        # restrictions depend on distancing of agent and other
        if other.distancing:
            
            # all positions within 1 space of other are unavailble
            other_adj = self.get_adj_positions(other.position)
        
            # remove restricted positions
            for pos in other_adj:
                if pos in avail:
                    avail.remove(pos)
    
    def remove_positions_distancing_self(self, agent, other, avail):
        '''
        If this agent is distancing, removes all positions
        which would breach distancing due to proximity
        of other agent.
        '''
        
        if agent.distancing:
            
            # all positions within 1 space of other are unavailable
            other_adj = self.get_adj_positions(other.position)
        
            # remove restricted positions
            for pos in other_adj:
                if pos in avail:
                    avail.remove(pos)
        
           
    def get_available_positions(self, agent, agents):
        '''
        Returns a list of possible positions to which a
        given agent can move, given the other agents.
        
        Constrained by the current position of other agents
        and whether another agent is distancing.
        
        determine which of those positions are:
         1 - not occupied
         2 - allow this agent to maintain its own distancing boundaries if necessary
         3 - outside distancing boundaries of another agent
        '''               
        # if no nearby agents, all positions available
        avail = self.get_adj_positions(agent.position)
                
        # find all nearby agents
        nearby = self.find_nearby_agents(agents, agent.position)

        # if there are nearby agents, determine availability
        if nearby:

            # check moves against each nearby agent
            for other in nearby:

                # remove occupied positions
                self.remove_position_if_occupied(other, avail)
                
                # remove positions which breach distancing of others
                self.remove_positions_distancing_others(other, avail)
                
                # remove positions which breach distancing of this agent
                self.remove_positions_distancing_self(agent,other,avail)
                    
        return avail      
        
       
    def move_agent(self, agent, positions):
        '''
        Selects a random position from positions and 
        updates the agent to be at that location.
        
        If positions list is empty, does not move
        the agent.
        '''
        if positions:
            i = random.randint(0,len(positions)-1) # random index
            agent.position = list(positions)[i]     
        
      
    def move_all_agents(self, agents):
        '''
        Selects and moves each agent in agents.
        '''
        for agent in agents:
            moves = self.get_available_positions(agent, agents)
            self.move_agent(agent, moves)
            
            
    def check_position_conflicts(self, agents):
        '''
        Returns a set of positions which contain
        more than one agent.
        '''
        
        conflicts = set()
        chosen = dict()
        for agent in agents:
            pos = agent.position
            if pos in chosen:
                chosen[pos] +=1
            else:
                chosen[pos] = 1
        
        for key in chosen.keys():
            if chosen[key] > 1:
                conflicts.add(key)
         
        return conflicts
        
        
    def check_distancing_conflicts(self, agents):
        '''
        Returns a set of positions which contain
        agents that breach distancing constraints.
        '''
        
        conflicts = set()
        
        for agent in agents:
            
            if agent.distancing:
                                
                pos = agent.position
                adj = self.get_adj_positions(pos)
                
                # no other agent should be in an adjacent position
                
                # Note agent won't conflict with itself because its
                # position is not in adj
                for each in agents:
                    
                    # check whether any agent is within the distancing range
                    if each.position in adj:
                        conflicts.add(each.position)
        
        return conflicts

#### Test AgentMover class

In [316]:
print('Test get_random_position()')
n=25
am = AgentMover(n)

selected = list()
position = am.get_random_position(selected)

# check if position is within n x n grid
assert position[0] >= 0
assert position[0] < n
assert position[1] >= 0
assert position[1] < n

# check if position has been taken already
# create list with all positions taken except (0,0)
selected = [(i,j) for i in range(n) for j in range(n) if (i,j) != (0,0) ]
position = am.get_random_position(selected)
assert position == (0,0)

Test get_random_position()


In [317]:
print('Test split_agents()')
n=25
am = AgentMover(n)

a1 = Agent((1,2), 'I', mask=True, distancing=True)
a2 = Agent((1,2), 'I', mask=True, distancing=True)
a3 = Agent((1,2), 'R', mask=True, distancing=False)
a4 = Agent((1,2), 'R', mask=True, distancing=False)
a5 = Agent((1,2), 'R', mask=True, distancing=False)

distancing_agents, other_agents = am.split_agents([a1,a2,a3,a4,a5])

for agent in distancing_agents:
    assert agent.status == 'I'
    
for agent in other_agents:
    assert agent.status == 'R'
    
assert len(distancing_agents) == 2
assert len(other_agents) == 3

Test split_agents()


In [318]:
print('Test get_x()')

n=25
am = AgentMover(n)

# move 1 space
assert am.get_x(0,-1) == n-1
assert am.get_x(0,1) == 1
assert am.get_x(n-1,1) == 0

# move 2 spaces
assert am.get_x(0,-2) == n-2
assert am.get_x(0,2) == 2
assert am.get_x(n-1,2) == 1

Test get_x()


In [319]:
print('Test get_y()')

n=25
am = AgentMover(n)

# move 1 space
assert am.get_y(0,-1) == n-1
assert am.get_y(0,1) == 1
assert am.get_y(n-1,1) == 0

# move 2 spaces
assert am.get_y(0,-2) == n-2
assert am.get_y(0,2) == 2
assert am.get_y(n-1,2) == 1

Test get_y()


In [320]:
print('Test get_adj_positions()')

n=25
am = AgentMover(n)

# non-edge position
current_position = (1,1)
adjacent_correct = [(0,0),(0,1),(0,2),
                    (1,0),(1,2),
                    (2,0),(2,1),(2,2)]
adjacent = am.get_adj_positions(current_position)
print(adjacent)
for each in adjacent_correct:    
    assert each in adjacent, str(each) + ' is missing from result.'


# lower right corner
current_position = (24,24)
adjacent_correct = [(23,0),(23,23),(23,24),
                    (24,0),(24,23),
                    (0,0),(0,23),(0,24)]
adjacent = am.get_adj_positions(current_position)
print(adjacent)
for each in adjacent_correct:    
    assert each in adjacent, str(each) + ' is missing from result.'

# upper right corner
current_position = (0,24)
adjacent_correct = [(24,0),(24,23),(24,24),
                    (0,0),(0,23),
                    (1,0),(1,23),(1,24)]
adjacent = am.get_adj_positions(current_position)
print(adjacent)
for each in adjacent_correct:    
    assert each in adjacent, str(each) + ' is missing from result.'

# lower left corner
current_position = (24,0)
adjacent_correct = [(23,0),(23,1),(23,24),
                    (24,1),(24,24),
                    (0,0),(0,1),(0,24)]
adjacent = am.get_adj_positions(current_position)
print(adjacent)
for each in adjacent_correct:    
    assert each in adjacent, str(each) + ' is missing from result.'

# upper left corner
current_position = (0,0)
adjacent_correct = [(24,0),(24,1),(24,24),
                    (0,1),(0,24),
                    (1,0),(1,1),(1,24)]
adjacent = am.get_adj_positions(current_position)
print(adjacent)
for each in adjacent_correct:    
    assert each in adjacent, str(each) + ' is missing from result.'

Test get_adj_positions()
{(0, 1), (1, 2), (2, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0)}
{(0, 23), (23, 23), (24, 23), (0, 0), (23, 0), (24, 0), (0, 24), (23, 24)}
{(0, 23), (24, 23), (0, 0), (1, 24), (24, 0), (1, 23), (1, 0), (24, 24)}
{(0, 1), (23, 1), (24, 1), (0, 0), (23, 0), (0, 24), (23, 24), (24, 24)}
{(0, 1), (24, 1), (1, 24), (1, 1), (24, 0), (1, 0), (0, 24), (24, 24)}


In [321]:
print('Test positions_available()')

n=1
am = AgentMover(n)
a1 = Agent((-1,-1), 'I', mask=True, distancing=True)
unavailable = set()

# (unavailable, agents, positioned)
assert am.positions_available(unavailable,[a1],0) == True 

unavailable.add(1)
assert am.positions_available(unavailable,[a1],0) == False


Test positions_available()


In [322]:
print('Test position_distancing_agents()')

# only possible to place one agent
n=3
am = AgentMover(n)
agents = [Agent((-1,-1), 'I', mask=True, distancing=True) for i in range(2)]
result, unavailable = am.position_distancing_agents(agents)
assert result == False


# multiple agents can be placed
n=25
am = AgentMover(n)
agents = [Agent((-1,-1), 'I', mask=True, distancing=True) for i in range(2)]
result, unavailable = am.position_distancing_agents(agents)
assert result == True


Test position_distancing_agents()


In [323]:
print('Test position_remaining_agents()')

n=1
am = AgentMover(n)

a1 = Agent((-1,-1), 'I', mask=True, distancing=False)
a2 = Agent((-1,-1), 'I', mask=True, distancing=False)

unavailable = set()

assert am.position_remaining_agents([a1],unavailable) == True


unavailable.add((0,0))
assert am.position_remaining_agents([a1],unavailable) == False

Test position_remaining_agents()


In [347]:
print('Test position_agents()')

# success
n=25
am = AgentMover(n)
a1 = Agent((-1,-1), 'I', mask=True, distancing=True)
a2 = Agent((-1,-1), 'I', mask=True, distancing=False)
agents = [a1,a2]

assert am.position_agents(agents) == True
for each in agents:
    print(each)
    assert each.position != (-1,-1)
    
# failure
n=1
am = AgentMover(n)
a1 = Agent((-1,-1), 'I', mask=True, distancing=True)
a2 = Agent((-1,-1), 'I', mask=True, distancing=False)
agents = [a1,a2]

try:
    am.position_agents(agents)
    assert False
except ValueError:
    assert True
    
# try with full number of Agents with no distancing
agents = [Agent((-1,-1), 'I', mask=True, distancing=False) for i in range(250)]


Test position_agents()
Agent{position:(13, 20), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}
Agent{position:(15, 12), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:True}


250

In [325]:
print('Test get_search_space()')
n=25
am = AgentMover(n)
   

# test 3 diameter, no wrap
position = (5,5)

expected = [(2, 2),(2, 3),(2, 4),(2, 5),(2, 6),(2, 7),(2, 8),
            (3, 2),(3, 3),(3, 4),(3, 5),(3, 6),(3, 7),(3, 8),
            (4, 2),(4, 3),(4, 4),(4, 5),(4, 6),(4, 7),(4, 8),
            (5, 2),(5, 3),(5, 4),       (5, 6),(5, 7),(5, 8),
            (6, 2),(6, 3),(6, 4),(6, 5),(6, 6),(6, 7),(6, 8),
            (7, 2),(7, 3),(7, 4),(7, 5),(7, 6),(7, 7),(7, 8),
            (8, 2),(8, 3),(8, 4),(8, 5),(8, 6),(8, 7),(8, 8)]
actual = am.get_search_space(position,am.directions_dia_3)
for each in expected:
    assert each in actual, 'missing ' + str(each)
    
    
# test 3 diameter, wrap
position = (0,0)

expected = [       (0, 1),(0, 2),(0, 3),       (0, 22),(0, 23),(0, 24),
            (1, 0),(1, 1),(1, 2),(1, 3),       (1, 22),(1, 23),(1, 24),
            (2, 0),(2, 1),(2, 2),(2, 3),       (2, 22),(2, 23),(2, 24),
            (3, 0),(3, 1),(3, 2),(3, 3),       (3, 22),(3, 23),(3, 24),
            
            (22, 0),(22, 1),(22, 2),(22, 3),   (22, 22),(22, 23),(22, 24),
            (23, 0),(23, 1),(23, 2),(23, 3),   (23, 22),(23, 23),(23, 24),
            (24, 0),(24, 1),(24, 2),(24, 3),   (24, 22),(24, 23),(24, 24)]

actual = am.get_search_space(position,am.directions_dia_3)
for each in expected:
    assert each in actual, 'missing ' + str(each)
    


Test get_search_space()


In [326]:
print('Test find_nearby_agents()')
n=25
am = AgentMover(n)
position = (5,5)

# no agent nearby
a1 = Agent((1,1), 'I', mask=True, distancing=False)
result = am.find_nearby_agents([a1], position)
assert len(result) == 0
print(result)
print()

# one agent nearby
a2 = Agent((6,6), 'I', mask=True, distancing=False)
result = am.find_nearby_agents([a1,a2], position)
assert len(result) == 1
for each in result:
    print(each)
print()

# two agents nearby
a3 = Agent((6,7), 'I', mask=True, distancing=False)
result = am.find_nearby_agents([a1,a2,a3], position)
assert len(result) == 2
for each in result:
    print(each)
print()
    
# three agents nearby, wrapped
position = (0,0)
a4 = Agent((24,24), 'I', mask=True, distancing=False)
a5 = Agent((23,23), 'I', mask=True, distancing=False)
a6 = Agent((22,22), 'I', mask=True, distancing=False)
result = am.find_nearby_agents([a4,a5,a6], position)
assert len(result) == 3
for each in result:
    print(each)
print()

Test find_nearby_agents()
[]

Agent{position:(6, 6), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:False}

Agent{position:(6, 6), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:False}
Agent{position:(6, 7), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:False}

Agent{position:(24, 24), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:False}
Agent{position:(23, 23), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:True}
Agent{position:(22, 22), status:I, mask:True, distancing:False, days_infected:0, asymptomatic:False}



In [327]:
print('Test remove_position_if_occupied()')
n=25
am = AgentMover(n)
available = [(0,0),(1,1)]

# attempt where agent is in available
a1 = Agent((0,0), 'I', mask=True, distancing=True)
am.remove_position_if_occupied(a1, available)

assert (0,0) not in available

# attempt where position not in available
a2 = Agent((1,0), 'I', mask=True, distancing=True)
am.remove_position_if_occupied(a2, available)

assert len(available) == 1


Test remove_position_if_occupied()


In [328]:
print('Test remove_positions_distancing_others()')

n=25
am = AgentMover(n)
a1 = Agent((5,5), 'I', mask=True, distancing=False)
available = [(4, 4), (4, 5), (4, 6), 
             (5, 4),         (5, 6), 
             (6, 4), (6, 5), (6, 6)]



# other agent not distancing
a3 = Agent((3,4), 'I', mask=True, distancing=False)
am.remove_positions_distancing_others(a3, available)
assert len(available) == 8


# other agent distancing
a2 = Agent((3,4), 'I', mask=True, distancing=True)
removed = [(4, 4), (4, 5)]

am.remove_positions_distancing_others(a2, available)
for each in removed:
    assert each not in available

Test remove_positions_distancing_others()


In [329]:
print('Test remove_positions_distancing_self()')

n=25
am = AgentMover(n)
a1 = Agent((5,5), 'I', mask=True, distancing=True)
available = [(4, 4), (4, 5), (4, 6), 
             (5, 4),         (5, 6), 
             (6, 4), (6, 5), (6, 6)]



# other agent not distancing
a3 = Agent((4,8), 'I', mask=True, distancing=False)
am.remove_positions_distancing_self(a1, a3, available)
assert len(available) == 8


# other agent distancing
a2 = Agent((3,4), 'I', mask=True, distancing=False)
removed = [(4, 4), (4, 5)]

am.remove_positions_distancing_self(a1, a2, available)
for each in removed:
    assert each not in available

Test remove_positions_distancing_self()


In [330]:
print('Test get_available_positions()')

n=25
am = AgentMover(n)
a1 = Agent((5,5), 'I', mask=True, distancing=False)
expected = [(4, 4), (4, 5), (4, 6), 
            (5, 4),         (5, 6), 
            (6, 4), (6, 5), (6, 6)]

# no nearby agents
a2 = Agent((0,0), 'I', mask=True, distancing=False)
available = am.get_available_positions(a1, [a2])
assert len(available)==8


# 1 agent occupies possible move
a2 = Agent((4,4), 'I', mask=True, distancing=False)
available = am.get_available_positions(a1, [a2])
assert (4,4) not in available

# 2 agents occupy possible moves
a3 = Agent((6,6), 'I', mask=True, distancing=False)
available = am.get_available_positions(a1, [a2,a3])
assert (4,4) not in available and (6,6) not in available

# 1 other agent distancing
a4 = Agent((3,4), 'I', mask=True, distancing=True)
available = am.get_available_positions(a1, [a4])
assert (4,4) not in available and (4,5) not in available

# this agent distancing
a5 = Agent((3,4), 'I', mask=True, distancing=False)
a1.distancing = True
available = am.get_available_positions(a1, [a5])
assert (4,4) not in available and (4,5) not in available

# all spaces restricted
a6 = Agent((3,4), 'I', mask=True, distancing=True)
a7 = Agent((4,7), 'I', mask=True, distancing=True)
a8 = Agent((7,6), 'I', mask=True, distancing=True)
a9 = Agent((6,3), 'I', mask=True, distancing=True)
available = am.get_available_positions(a1, [a6,a7,a8,a9])
assert len(available) == 0


Test get_available_positions()


In [331]:
print('Test move_agent()')

n=25
am = AgentMover(n)
a1 = Agent((1,2), 'I', mask=True, distancing=True)

# no movement because list is empty
am.move_agent(a1, [])

assert a1.position == (1,2)


# movement to only available position
am.move_agent(a1, [(1,1)])

assert a1.position == (1,1)


# movement to random available position
pos = [(1,1),(1,3),(2,2)]

num = 10000
counter = 0
for i in range(num):
    am.move_agent(a1, pos)
    if a1.position == (2,2):
        counter += 1
assert round(counter/num,2) == 0.33, 'rate ' + str(round(counter/num,2)) + ' is not 0.33'

Test move_agent()


In [332]:
print('Test check_position_conflicts()')

n=25
am = AgentMover(n)

a1 = Agent((5,5), 'I', mask=True, distancing=True)
a2 = Agent((0,0), 'I', mask=True, distancing=True)

conflicts = am.check_position_conflicts([a1,a2])
assert len(conflicts) == 0


a1 = Agent((5,5), 'I', mask=True, distancing=True)
a2 = Agent((5,5), 'I', mask=True, distancing=True)

conflicts = am.check_position_conflicts([a1,a2])
assert (5,5) in conflicts

Test check_position_conflicts()


In [333]:
print('Test check_distancing_conflicts()')

n=25
am = AgentMover(n)

a1 = Agent((1,1), 'I', mask=True, distancing=True)
a2 = Agent((5,5), 'I', mask=True, distancing=True)

# not within distancing constraints
conflicts = am.check_distancing_conflicts([a1,a2])
assert len(conflicts) == 0

# within distancing constraints
a1.position = (4,4)
conflicts = am.check_distancing_conflicts([a1,a2])
assert (4,4) in conflicts and (5,5) in conflicts
print(conflicts)

Test check_distancing_conflicts()
{(4, 4), (5, 5)}


In [334]:
print('Test move_all_agents()')

n=25
am = AgentMover(n)

# example where there is no conflict for two agents
a1 = Agent((5,5), 'I', mask=True, distancing=True)
a2 = Agent((0,0), 'I', mask=True, distancing=True)

available_a1 = [(4, 4), (4, 5), (4, 6), 
                (5, 4),         (5, 6), 
                (6, 4), (6, 5), (6, 6)]

available_a2 = [        (0, 1),      (0, 24), 
                (1, 0), (1, 1),      (1, 24), 
                
                (24, 0), (24, 1),    (24, 24)]

am.move_all_agents([a1,a2])
assert a1.position in available_a1
assert a2.position in available_a2


# example where this is potential conflict for two agents

a1.position = (5,5)
a6 = Agent((3,4), 'I', mask=True, distancing=True)
a7 = Agent((4,7), 'I', mask=True, distancing=True)
a8 = Agent((7,6), 'I', mask=True, distancing=True)
a9 = Agent((7,3), 'I', mask=True, distancing=True)

agents = [a1,a6,a7,a8,a9]

# check a large number of iterations
# randomize order in list so different agents are moved first each time
for i in range(1000):
    
    # reset positions
    a1.position = (5,5)
    a6.position = (3,4)
    a7.position = (4,7)
    a8.position = (7,6)
    a9.position = (7,3)
    
    # randomize order
    random.shuffle(agents) 
    am.move_all_agents(agents)

    # check that no agents are in the same position
    conflicts_pos = am.check_position_conflicts(agents)
    assert len(conflicts_pos) == 0
        
    # check that no agents breach distancing
    conflicts_dist = am.check_distancing_conflicts(agents)
    assert len(conflicts_dist) == 0

# print last iteration positions for visual check
for each in agents:
    print(each)

Test move_all_agents()
Agent{position:(2, 3), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}
Agent{position:(4, 5), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}
Agent{position:(5, 7), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}
Agent{position:(8, 3), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}
Agent{position:(7, 7), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}


In [335]:
print('All tests pass for AgentMover class.')

All tests pass for AgentMover class.


#### Define ABM class

In [337]:
class ABM:


    status_colors = {'R': 'r', 'S': 'b', 'I': 'g', 'Q': 'k', 'D': 'm'}

    def __init__(self, n, m, num_infected, percent_distancing, percent_mask, percent_vaccinated = 0.0):
        self.n = n
        self.m = m
        
        # validate
        if m > n*n: 
            raise ValueError('n x n grid cannot hold all agents')

        # generate agents
        ag = AgentGenerator(self.m, num_infected, percent_distancing, percent_mask, percent_vaccinated)
        self.agents = ag.generate_agents()
        
        # position agents
        am = AgentMover(self.n)
        am.position_agents(self.agents)
        
        # set baseline metrics
        self.counts = [] # holds each dictionary of captured metrics
        self.num_recovered = 0
        self.num_infected = 0
        self.num_susceptible = 0 
        self.num_dead = 0       
        self.num_quarantine = 0
        
   
    def count_baseline_metrics(self):
        '''
        Counts current values for baseline metrics.
        '''
        for agent in self.agents:
            if agent.status == 'I':
                self.num_infected += 1
            elif agent.status == 'S':
                self.num_susceptible += 1
            elif agent.status == 'R':
                self.num_recovered += 1
            elif agent.status == 'Q':
                self.num_quarantine += 1
         
        
    def remove_agent(self, agent):
        '''
        Remove agent from simulation if agent has died
        and increments num_dead by 1.
        '''
        self.agents.remove(agent)
        self.num_dead += 1
      
    
    def add_counts(self):
        '''
        Generates a dictionary of the current metrics and adds
        the dictionary to the list of counts.
        '''
        counts = {'R': self.num_recovered, 'D': self.num_dead, 'I': self.num_infected, 'S': self.num_susceptible, 'Q': self.num_quarantine}
        self.counts.append(counts)       
            
    
    def get_adj_agents(self, agent):
        '''
        Returns a list of other Agents which are within
        1 position of this Agent.
        '''
        
        nearby = []
        
        am = AgentMover(self.n)
        positions = am.get_adj_positions(agent.position)

        for each in self.agents:
            if each.position in positions:
                nearby.append(each) # found one
    
        return nearby
            
    
        
    def run_simulation(self, num_steps):
        
        self.count_baseline_metrics()
        am = AgentMover(self.n)
        
        # update agents and metrics for each time step
        for t in range(num_steps):
            
            # update agent properties
            for agent in self.agents:
                
                if agent.has_died():
                    self.remove_agent(agent)
                else:
                    adj = self.get_adj_agents(agent)
                    agent.update_agent(adj)
            
            # move all agents
            am.move_all_agents(self.agents)

            #?? capture metrics after every agent or after every time step   
            self.add_counts() 

        return self.counts
    
    
    
        

#### Test ABM class

In [338]:
print('Test run_simulation()')

n = 25
m = 250
num_infected = 10
percent_distancing = 0.25
percent_mask = 0.35
percent_vaccinated = 0.1
abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)

counts = abm.run_simulation(10)
assert len(counts == 10)

Test run_simulation()


ValueError: Unable to find positions for remaining agents

In [111]:
print('Test constructor')

# raises exception

n = 3
m = 250
num_infected = 10
percent_distancing = 0.25
percent_mask = 0.35
percent_vaccinated = 0.1

try:
    abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)
    assert False
except ValueError:
    assert True
    
# doesn't raise exception
n = 25
abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)

# check counts of Agents
assert len(abm.agents) == m

# ensure all Agents were placed
for agent in abm.agents:
    assert agent.position != (-1,-1)

Test constructor


In [344]:
print('Test count_baseline_metrics()')

n = 25
m = 250
num_infected = 10
percent_distancing = 0.25
percent_mask = 0.35
percent_vaccinated = 0.1
abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)

# zero counts
assert abm.num_recovered == 0
assert abm.num_infected == 0
assert abm.num_susceptible == 0
assert abm.num_dead == 0
assert abm.num_quarantine == 0


# after performing counts
abm.count_baseline_metrics()
assert abm.num_recovered == 25, abm.num_recovered 
assert abm.num_infected == 10, abm.num_infected 
assert abm.num_susceptible == m - abm.num_recovered - abm.num_infected, abm.num_susceptible
assert abm.num_dead == 0, abm.num_dead
assert abm.num_quarantine == 0, abm.num_quarantine


Test count_baseline_metrics()


ValueError: Unable to find positions for other agents

In [116]:
print('Test remove_agent()')

n = 25
m = 250
num_infected = 10
percent_distancing = 0.25
percent_mask = 0.35
percent_vaccinated = 0.1
abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)

a = abm.agents[0] # select an agent from agents

abm.remove_agent(a)

assert a not in abm.agents
assert len(abm.agents) == m-1, len(agents)
assert abm.num_dead == 1

Test remove_agent()


In [124]:
print('Test add_counts()')

n = 25
m = 250
num_infected = 10
percent_distancing = 0.25
percent_mask = 0.35
percent_vaccinated = 0.1
abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)
abm.count_baseline_metrics()

# empty counts list
assert len(abm.counts) == 0


# add set of metrics to list
abm.add_counts()
assert len(abm.counts) == 1

for count_dict in abm.counts:
    print(count_dict)
    assert count_dict['R'] == 25
    assert count_dict['D'] == 0
    assert count_dict['I'] == 10
    assert count_dict['Q'] == 0
    assert count_dict['S'] == 250 - 25 - 10
    

Test add_counts()
{'R': 25, 'D': 0, 'I': 10, 'S': 215, 'Q': 0}


In [144]:
print('Test get_adj_agents()')

n = 25
m = 250
num_infected = 10
percent_distancing = 0.25
percent_mask = 0.35
percent_vaccinated = 0.1
abm = ABM(n,m,num_infected, percent_distancing, percent_mask, percent_vaccinated)

a1 = Agent((1,1), 'I', mask=True, distancing=False)
a2 = Agent((0,0), 'I', mask=True, distancing=False)
a3 = Agent((5,5), 'I', mask=True, distancing=False)


# no agents nearby Agent
abm.agents = [a1,a3]
assert len(abm.get_adj_agents(a1)) == 0


# at least one agent nearby Agent
abm.agents = [a1,a2,a3]
adj = abm.get_adj_agents(a1)

assert len(adj) == 1
assert a2 in adj



Test get_adj_agents()


In [ ]:
#  def runAndPlotSimulations(num_simulations, num_steps, world_size, num_agents, num_infected, percent_distancing, percent_mask, percent_vaccinated):
#     sim_counts = []
#     for i in range(num_simulations):
#       model = ABM(world_size, num_agents, num_infected, percent_distancing, percent_mask, percent_vaccinated)
#       counts = model.runSimulation(num_steps)
#       sim_counts.append(counts)


#     statusColors = {'R': 'r', 'S': 'b', 'I': 'g', 'Q': 'k', 'D': 'm'}
#     fig, ax = plt.subplots(figsize=(20, 10))
#     x = list(range(num_steps))

#     for status in ['R', 'S', 'I', 'Q', 'D']:
#       avgs = []
#       errs = []
#       for j in range(num_steps):
#         stats = np.array([counts[j][status] for counts in sim_counts])
#         avgs.append(np.mean(stats))
#         errs.append(np.std(stats))

#       ax.errorbar(x,avgs,yerr=errs, fmt=statusColors[status], label=status)

#     ax.legend(loc="upper left", bbox_to_anchor=(1, 0.5))
#     plt.show()

## 1.2 Run 10 simulations, each of which tracks 250 agents over 365 days in a 25 x 25 world with the following initial conditions: 

* 5 infected agents (I), the rest are susceptible (S)
* 0% of agents wear masks
* 0% of agents practice physical distancing

Keep track of the number of S, I,Q,R, and D agents over time in each simulation. Plot the averages of these trajectories with standard error bars. 
What is the final value of R?
What is the peak number of active cases (I+Q)? 
How many days does it take for the virus to go extinct (i.e. I+Q=0)? 


In [ ]:
# runAndPlotSimulations(10, 365, 25, 250, num_infected, percent_distancing, percent_mask, percent_vaccinated)

## 1.3 Run 10 simulations, each of which tracks 250 agents over 365 days in a 25 x 25 world with the following initial conditions: 


*   5 infected agents (I), the rest are susceptible (S)
*   p% of agents wear masks
*   p% of agents practice physical distancing

Keep track of the number of S, I,Q,R, and D agents over time in each simulation. Plot the averages of these trajectories with standard error bars. 
What value of p reduces the final value of R to roughly half of the final value of R you obtained for part 1.2?
What is the final value of R?
What is the peak number of active cases (I+Q)? 
How many days does it take for the virus to go extinct (i.e. I+Q=0)? 

In [ ]:
# runAndPlotSimulations(10, 365, 25, 250, num_infected, percent_distancing, percent_mask, percent_vaccinated)

## 1.4 Run 10 simulations, each of which tracks 250 agents over 365 days in a 25 x 25 world with the following initial conditions: 


*   5 infected agents (I), p% of agents have been vaccinated (i.e., start in state R), the rest are susceptible (S)
*   0% of agents wear masks
*   0% of agents practice physical distancing


Keep track of the number of S, I,Q,R, and D agents over time in each simulation. Plot the averages of these trajectories with standard errors. 
What value of p reduces the peak number of active cases (I+Q) to roughly half of the final value of (I+Q) you obtained for part 1.2?
What is the final value of R?
What is the peak number of active cases (I+Q)? 
How many days does it take for the virus to go extinct (i.e. I+Q=0)? 

In [ ]:
# runAndPlotSimulations(10, 365, 25, 250, num_infected, percent_distancing, percent_mask, percent_vaccinated)